*change in version 6: change in proc_df*

* [DataFrames](#section-one)
* [CT Images](#section-two)
* [LightGBM](#section-three)
* [Submission](#section-four)

<a id="section-one"></a>
# DataFrames

First, let's import the necessary modules.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns

from pydicom import dcmread
import cv2

path = "/kaggle/input/osic-pulmonary-fibrosis-progression/"

Now, let's have a look at the files:

In [ ]:
ls /kaggle/input/osic-pulmonary-fibrosis-progression/

In [ ]:
ls /kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430

We have csv files for train, test datasets, and their image files. Let's get the train.csv, test.csv, and sample_submission.csv:

In [ ]:
train_df  = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")
subm = pd.read_csv(path + "sample_submission.csv")

In [ ]:
train_df

We have 1549 images in the train dataset.

In [ ]:
train_df.Patient.nunique()

we have 176 patients in train

In [ ]:
train_df.Weeks.max()

In [ ]:
train_df.Weeks.min()

In [ ]:
fig, ax = plt.subplots(1,1)

sns.distplot(train_df[train_df["Weeks"].notna()]["Weeks"], ax=ax, color="#2222EE")
ax.set_title("distribution of weeks in train");

"Weeks" are between -5 and 133, and ths distribution of weeks is as in the above graph.

The FVC value distribution:

In [ ]:
fig, ax = plt.subplots(1,1)

sns.distplot(train_df[train_df["FVC"].notna()]["FVC"], ax=ax, color="#22EE22")
ax.set_title("distribution of FVC in train");

The Percent value distribution:

In [ ]:
fig, ax = plt.subplots(1,1)

sns.distplot(train_df[train_df["Percent"].notna()]["Percent"], ax=ax, color="#EE2222")
ax.set_title("distribution of Percent in train");

The Age value distribution:

In [ ]:
fig, ax = plt.subplots(1,1)

sns.distplot(train_df[train_df["Age"].notna()]["Age"], ax=ax, color="#992299")
ax.set_title("distribution of Age in train");

Sex distribution:

In [ ]:
train_df.Sex.value_counts()

In [ ]:
train_df.Sex.value_counts(normalize=True)

In [ ]:
train_df.groupby("Patient")["Sex"].first().value_counts(normalize=True)

Only 21% of the train images are Female.

In [ ]:
train_df["SmokingStatus"].value_counts()

In [ ]:
train_df["SmokingStatus"].value_counts(normalize=True)

Now, lets have a look at the test dataset:

In [ ]:
test_df

We have only five patients, all male, not any "currently smokes", all at age 68 and older. Let's merge submission and test files:

In [ ]:
def merge_subm_test(subm,test_df):
    
    a = subm['Patient_Week'].str.split("_", expand=True)
    a.columns=["Patient","Week"]
        
    test_df = test_df.merge(a, on="Patient")

    return test_df

test_df = merge_subm_test(subm,test_df)

In [ ]:
test_df

In [ ]:
test_df.groupby(["Patient"])["Weeks"].count()

In [ ]:
test_df.groupby(["Patient"])["Week"].first()

In [ ]:
test_df.groupby(["Patient"])["Week"].last()

We have been asked for the estimations of 146 weeks for each patient, starting with -12 and ending with 133.

<a id="section-two"></a>
# CT Images

Let's have a look at a patient's CT scan at the base week (train dataset):

In [ ]:
ls /kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/

In [ ]:
fig, axs = plt.subplots(5, 6,figsize=(20,20))
for n in range(0,30):
    #print(int(n/6),np.mod(n,6))
    image = dcmread("/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/" + str(n+1) + ".dcm")
    axs[int(n/6),np.mod(n,6)].imshow(image.pixel_array);

Now, et's have a look at a patient's CT scan at the base week, in the test dataset:

In [ ]:
ls /kaggle/input/osic-pulmonary-fibrosis-progression/test/ID00419637202311204720264/

In [ ]:
fig, axs = plt.subplots(5, 6,figsize=(20,20))
for n in range(0,28):
    #print(int(n/6),np.mod(n,6))
    image = dcmread("/kaggle/input/osic-pulmonary-fibrosis-progression/test/ID00419637202311204720264/" + str(n+1) + ".dcm")
    axs[int(n/6),np.mod(n,6)].imshow(image.pixel_array);

<a id="section-three"></a>
# LightGBM

Let's arrange dataframes for LightGBM:

In [ ]:
train_df  = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")
subm = pd.read_csv(path + "sample_submission.csv")

In [ ]:
def proc_df(df):
    
    df = pd.concat([df, pd.get_dummies(df["SmokingStatus"], dtype=int)], axis=1)
    df.drop(["SmokingStatus"],axis=1,inplace=True)

    df['Sex'] = df['Sex'].map({'Female': 0, 'Male': 1})

    return df

train_df = proc_df(train_df)

In [ ]:
def proc_train(df):

    df_final = pd.DataFrame()

    for patient, df2 in df.groupby('Patient'):
        
        df11 = df2[["Patient","Weeks","FVC"]]


        df2 = df2.rename(columns={
            "FVC": "base_FVC", 
            "Percent": "base_Percent", 
            "Weeks": "base_Week"
        }, errors="raise")
        
        df3 = pd.merge(df11, df2, how='outer', on='Patient')
        df3 = df3.query('Weeks!=base_Week')
        df3['week_diff'] = df3['base_Week'] - df3['Weeks']

        df_final = pd.concat([df_final, df3])
        
    return df_final.reset_index(drop=True)

In [ ]:
train_df = proc_train(train_df)

In [ ]:
a = subm['Patient_Week'].str.split("_", expand=True)
a.columns=["Patient","Weeks"]
a["Weeks"] = a["Weeks"].astype("int")
    
test_df.rename(
    columns={
        'Weeks': 'base_Week',
        'FVC': 'base_FVC',
        'Percent': 'base_Percent',
        'Age': 'Age'
    },
    inplace=True
)

test_df = proc_df(test_df)

test_df = pd.merge(a, test_df, how='left', on=['Patient'])
test_df['week_diff'] = test_df['base_Week'] - test_df['Weeks']

test_df

Let's define the column data types:

In [ ]:
train_df = train_df.drop(set(train_df.columns)-set(test_df.columns)-{'FVC', 'Percent'}, axis=1)
test_df = test_df.drop(set(test_df.columns)-set(train_df.columns), axis=1)

X = train_df.drop(["Patient","FVC"], axis=1)
y = train_df["FVC"]
test = test_df.drop(["Patient"], axis=1)

In [ ]:
X

In [ ]:
test

In [ ]:
num_fold = 5

def get_lgbm_model(X_train, y_train, X_val, y_val, fold, param_choice, columns):
    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    if param_choice == "normal":
        params = {
            "metric":"rmse"
        }
    elif param_choice == "quantile1":
        params = {
            "objective":"quantile",
            "alpha":0.2,
            "metric":"quantile"
        }    
    elif param_choice == "quantile2":
        params = {
            "objective":"quantile",
            "alpha":0.8,
            "metric":"quantile"
        }    
    
    model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
    model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], 
        verbose=1000, 
        early_stopping_rounds=100
    )

    fold_importance = pd.DataFrame()
    print(columns)
    print(model.feature_importances_)
    fold_importance["feature"] = columns
    fold_importance["importance"] = model.feature_importances_
    #plot_save_feat_imp(fold_importance, fold)    
    fold_importance = fold_importance.sort_values(by=['importance'])
    fold_importance.to_csv('feature_importances_'+ y_train.name + "_" + param_choice + str(fold) + '.csv')
    
    return model

def get_lgbm_pred(X, y, test, param_choice):
    print("get_lgbm_pred ", param_choice)

    pred = []
    pred_val = np.zeros((len(X)))
            
    #X_train, X_val, y_train, y_val = train_test_split(X_scaled, y.fillna(0).values, test_size=0.2, shuffle=True, random_state=42)
    kf = KFold(n_splits=num_fold, random_state=None, shuffle=False)
    fold = 0
    score = 0
    for train_index, test_index in kf.split(X, y):
        fold += 1
        print("fold ", fold)
    
        X_train = X.iloc[train_index, :]
        X_val = X.iloc[test_index, :]
        y_train = y[train_index]
        y_val = y[test_index]
        
        model = get_lgbm_model(X_train, y_train, X_val, y_val, fold, param_choice, X_train.columns)


        if fold ==1:
            pred.append(model.predict(test))
        else:
            pred += model.predict(test)

        
        pred_val[test_index] = model.predict(X_val)            
        score = score + np.sqrt(mean_squared_error(y_val, pred_val[test_index]))
        print("score ", str(score/fold))
            
                    
    print("\n\n\n")

    f = open("score","a+")
    f.write(str(score/num_fold)+", ")
    f.close()
    
    return pred[0]/num_fold, pred_val

Let's take "Confidence" as the difference between 0.1 and 0.9 quantiles:

In [ ]:
pred_FVC_te, pred_FVC_tr = get_lgbm_pred(X, y, test, "normal")

pred_FVC_te_q1, pred_FVC_tr_q1 = get_lgbm_pred(X, y, test, "quantile1")
pred_FVC_te_q2, pred_FVC_tr_q2 = get_lgbm_pred(X, y, test, "quantile2")

pred_conf_tr = pred_FVC_tr_q2 - pred_FVC_tr_q1
pred_conf_te = pred_FVC_te_q2 - pred_FVC_te_q1

Calculate the score:

In [ ]:
def metric(confidence, fvc, pred_fvc):

    confidence = max(confidence, 70)
    delta = min(abs(fvc-pred_fvc), 1000)
    score = -(math.sqrt(2)*(delta/confidence)) - np.log(math.sqrt(2)*confidence)
        
    return score

def calc_score(confidence, fvc, pred_fvc):
    
    score = 0
    for n in range(len(confidence)):
        score += (metric(confidence[n], fvc[n], pred_fvc[n]))
        
    return score/len(train_df)

score = calc_score(pred_conf_tr, train_df.FVC.values, pred_FVC_tr)
print(score)

<a id="section-four"></a>
# Submission

And finally the submission file:

In [ ]:
subm["FVC"] = pred_FVC_te
subm["Confidence"] = pred_conf_te
subm.to_csv("submission.csv", index=False)

In [ ]:
subm